In [ ]:
import pandas as pd

import random

f= "accepted_2007_to_2018Q4.csv"

num_lines = sum(1 for i in open(f))

num_lines

size= 2_200_000

size

ids= random.sample(range (1, num_lines),size)

len(ids)

df= pd.read_csv(f, skiprows= ids)

df.head()

In [ ]:
import pandas as pd
import random

f = "accepted_2007_to_2018Q4.csv"

def limpieza_data():
    num_lines = sum(1 for i in open(f))

    size = 2_200_000
    ids = random.sample(range(1, num_lines), size)

    column_types = {
        '0': 'str',      
        '19': 'float',
        '59': 'str',
        '118': 'str',
        '129': 'float',
        '130': 'float',
        '131': 'str',
        '134': 'str',
        '135': 'float',
        '136': 'str',
        '139': 'float',
    }

    df = pd.read_csv(f, skiprows=ids, low_memory=False, dtype=column_types)

    # Limpieza de NaNs
    num = 150000
    num_nans = df.isnull().sum()
    columns_menos_nans = num_nans[num_nans < num].index
    df_sin_nans = df[columns_menos_nans]

    # Selección de columnas útiles
    df_limpio = df_sin_nans[[
        "id",
        "loan_amnt",
        "funded_amnt",
        "term",
        "int_rate",
        "installment",
        "grade",
        "emp_length",
        "annual_inc",
        "dti",
        "fico_range_high",
        "fico_range_low",
        "revol_util",
        "loan_status",
        "out_prncp",
        "total_rec_prncp",
        "total_rec_int",
        "last_fico_range_high",
        "last_fico_range_low",
        "open_acc",
        "pub_rec",
        "inq_last_6mths",
        "purpose",
        "home_ownership",
        "recoveries",
        "num_tl_90g_dpd_24m",
        "delinq_2yrs"
    ]]

    return df_limpio

df_limpio = limpieza_data()


In [ ]:
df_limpio.to_csv("df_limpio_procesado.csv", index=False)


In [ ]:
print(df_limpio.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option("display.max_columns", None)
df_limpio

In [ ]:
print(df_limpio.shape)
print(df_limpio.dtypes)

In [ ]:
print(df_limpio.describe())

In [ ]:
print(df_limpio.isnull().sum())

In [ ]:
print(df_limpio[df_limpio["loan_amnt"].astype(str).str.contains("Total amount funded", na = False)])

df_limpio = df_limpio[~df_limpio["loan_amnt"].astype(str).str.contains("Total amount funded", na = False)]
print(df_limpio[df_limpio["loan_amnt"].astype(str).str.contains("Total amount funded", na = False)])

df_limpio["int_rate"] = df_limpio["int_rate"].astype(str).str.rstrip("%")
df_limpio["int_rate"] = pd.to_numeric(df_limpio["int_rate"], errors = "coerce") / 100

#Quitamos las dos ultimas filas del dataframe, ya que no nos sirven
df_limpio = df_limpio.drop(df_limpio.tail(2).index)

In [ ]:
columnas = ["loan_amnt", "int_rate", "annual_inc", "dti"]

limites = {
    "loan_amnt": None, 
    "int_rate": None, 
    "annual_inc": (0, 100000), 
    "dti": (0, 40) 
}

plt.figure(figsize=(14, 18))

for i, var in enumerate(columnas, 1):
    plt.subplot(2, 2, i)

    sns.violinplot(x="loan_status",
                   y=var,
                   data=df_limpio,
                   palette="pastel",
                   hue="loan_status",
                   legend=False)
    
    # Ajustar los límites del eje Y según la variable
    if limites[var] is not None:
        plt.ylim(limites[var])
    
    plt.xticks(rotation=90)
    plt.title(f"Distribución de {var} según loan_status")

plt.tight_layout()
plt.show()


In [ ]:
#en el dti podemos ver como al aumentar relacion deuda-ingreso hay mas impagos, cuando esta baja hay mas pagados
#con el anual_int no podemos ver tan clara esta relacion

In [ ]:
for i in columnas:
    plt.figure(figsize=(8, 4))
    plt.hist(df_limpio[i].dropna(), bins=30, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {i}', fontsize=14)
    plt.xlabel(i, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True)
    plt.show()

In [ ]:
columnas_categoricas = ["grade", "term", "purpose", "home_ownership", "loan_status"]

plt.figure(figsize=(16, 22))

for i, columna in enumerate(columnas_categoricas, 1):
    plt.subplot(3, 2, i)
    
    sns.countplot(data = df_limpio,
                  x = columna,
                  palette = "pastel",
                  hue = "loan_status")
    
    plt.title(f"Distribución de {columna} según loan_status")
    plt.xticks(rotation = 25)
    plt.legend(title = "loan_status",
               fontsize = 8,
               title_fontsize = 10,
               loc = "upper right")
    
plt.tight_layout()
plt.show()

In [ ]:
#Valores que no se han convertido bien
valores_invalidos = df_limpio[pd.to_numeric(df_limpio["int_rate"], errors = "coerce").isna()]
print(valores_invalidos)

In [ ]:
df_limpio["int_rate"] = df_limpio["int_rate"].astype(str)
df_limpio["int_rate"] = df_limpio["int_rate"].str.rstrip("%")
df_limpio["int_rate"] = pd.to_numeric(df_limpio["int_rate"], errors = "coerce") / 100

valores_invalidos = df_limpio[df_limpio["int_rate"].isna()]
print("Valores no convertidos correctamente (NaN):")
print(valores_invalidos)

In [ ]:
n_nans = df_limpio.isnull().sum().to_frame(name = "null_counts")
n_nans.reset_index(inplace=True)
n_nans.columns = ["columna", "numero_nans"]
n_nans #Para ver cuantos nans tiene cada columna

In [ ]:
# Limpieza de columnas
#Limpieza de las filas con nans en base a term, ya que no sirven
pd.set_option("display.max_rows", 30)
df_limpio = df_limpio.dropna(subset = ["term"])

# Columna term a term_months para dejar solo un int8
df_limpio["term_months"] = pd.to_numeric(df_limpio["term"].str.extract(r"(\d+)")[0], errors = "coerce").astype(np.int8)
df_limpio = df_limpio.drop(columns=["term"])

# Columna emp_length a emp_length_years para eliminar la cadena "years"
df_limpio["emp_length_years"] = df_limpio["emp_length"].str.replace(r"[^0-9+]", "", regex = True)
df_limpio = df_limpio.drop(columns=["emp_length"])

df_limpio

In [ ]:
df_numerico = df_limpio.select_dtypes(include = [np.number])

matriz_correlacion = df_numerico.corr()

plt.figure(figsize = (12, 8))

sns.heatmap(matriz_correlacion,
            annot = True,
            cmap = "coolwarm",
            fmt = ".2f")

plt.title("Matriz de correlación en base a loan_status")
plt.show()

In [ ]:
#aqui no se puede relacionar con las variables categoricas(entre ellas loan_status) creo que habria que quitarlo y hacerlo despues del onehotencoder y label encoder.

In [ ]:
plt.figure(figsize = (18, 6))
sns.scatterplot(data = df_limpio,
                x = "loan_amnt",
                y = "annual_inc",
                hue = "loan_status",
                palette = "pastel",
                alpha = 0.7)

plt.title("Relación entre loan_amnt y annual_inc según loan_status")
plt.xlabel("Monto del préstamo")
plt.ylabel("Ingreso anual")

plt.legend(title = "Estado del préstamo",
           loc = "upper right")

plt.show()

In [ ]:
import plotly.express as px

columnas_boxplot = ["loan_amnt", "annual_inc", "dti"]

for columna in columnas_boxplot:
    fig = px.box(df_limpio,
                 x = "loan_status",
                 y = columna,
                 title = f"Boxplot de {columna} según loan_status",
                 color = "loan_status",
                 color_discrete_sequence = px.colors.qualitative.Pastel)

    fig.show()

In [ ]:
# estos boxplots son lo mismo que los violines de arriba, los quitamos o lo dejamos pero hacemos que se vea mejor?